In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [13]:
import pandas as pd

In [14]:
raw = pd.read_csv('neo_v2.csv')

In [15]:
raw

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous,HAZARDOUS
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249220,54839744.08,Earth,False,16.73,False,0
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726660,61438126.52,Earth,False,20.00,True,1
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692100,49798724.94,Earth,False,17.83,False,0
3,3596030,(2012 BV13),0.096506,0.215794,24764.303140,25434972.72,Earth,False,22.20,False,0
4,3667127,(2014 GE35),0.255009,0.570217,42737.733760,46275567.00,Earth,False,20.09,True,1
...,...,...,...,...,...,...,...,...,...,...,...
90831,3763337,(2016 VX1),0.026580,0.059435,52078.886690,12300389.18,Earth,False,25.00,False,0
90832,3837603,(2019 AD3),0.016771,0.037501,46114.605070,54321206.42,Earth,False,26.00,False,0
90833,54017201,(2020 JP3),0.031956,0.071456,7566.807732,28400768.16,Earth,False,24.60,False,0
90834,54115824,(2021 CN5),0.007321,0.016370,69199.154480,68692060.53,Earth,False,27.80,False,0


In [16]:
raw_dropped = raw.drop(columns=['id','name','orbiting_body','sentry_object','hazardous'])

In [17]:
raw_dropped_mapped = raw_dropped.copy()

In [26]:
raw_data = np.loadtxt('raw_data.csv',delimiter=',',skiprows=1)

In [27]:
raw_data

array([[1.19827080e+00, 2.67941497e+00, 1.35692492e+04, 5.48397441e+07,
        1.67300000e+01, 0.00000000e+00],
       [2.65800000e-01, 5.94346868e-01, 7.35887267e+04, 6.14381265e+07,
        2.00000000e+01, 1.00000000e+00],
       [7.22029558e-01, 1.61450717e+00, 1.14258692e+05, 4.97987249e+07,
        1.78300000e+01, 0.00000000e+00],
       ...,
       [3.19561890e-02, 7.14562100e-02, 7.56680773e+03, 2.84007682e+07,
        2.46000000e+01, 0.00000000e+00],
       [7.32074000e-03, 1.63696720e-02, 6.91991545e+04, 6.86920605e+07,
        2.78000000e+01, 0.00000000e+00],
       [3.98616230e-02, 8.91332990e-02, 2.70244555e+04, 5.97721306e+07,
        2.41200000e+01, 0.00000000e+00]])

In [28]:
raw_input = raw_data[:,:-1]
raw_target = raw_data[:,-1]

In [32]:
len_raw = raw_input.shape[0]

In [33]:
num_one_target = int(sum(raw_target))
num_one_target

8840

In [36]:
zero_target_counter = 0
remove_indices = []

for i in range(len_raw):
    if raw_target[i] == 0:
        zero_target_counter += 1
        if zero_target_counter > num_one_target:
            remove_indices.append(i)

unscaled_inputs = np.delete(raw_input,remove_indices, axis=0)
unscaled_targets = np.delete(raw_target,remove_indices, axis=0)

In [37]:
targets_equal = unscaled_targets.copy()

In [38]:
scaled_inputs = preprocessing.scale(unscaled_inputs)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [49]:
shuffle_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffle_indices)
shuffled_inputs = scaled_inputs[shuffle_indices]
shuffled_targets = targets_equal[shuffle_indices]

### Split Data into train, validation, test data

In [50]:
len_data = int(shuffled_inputs.shape[0])

In [52]:
len_train = int(0.8*len_data)
len_validation = int(0.1*len_data)

In [53]:
train_inputs = shuffled_inputs[:len_train]
train_targets = shuffled_targets[:len_train]

In [54]:
validation_inputs = shuffled_inputs[len_train:len_train+len_validation]
validation_targets = shuffled_targets[len_train:len_train+len_validation]

In [55]:
test_inputs = shuffled_inputs[len_train+len_validation:]
test_targets = shuffled_targets[len_train+len_validation:]

In [59]:
print(np.sum(train_targets), len_train, np.sum(train_targets) / len_train)
print(np.sum(validation_targets), len_validation, np.sum(validation_targets) / len_validation)
print(np.sum(test_targets), test_targets.shape[0], np.sum(test_targets) / test_targets.shape[0])

7095.0 14144 0.5016261312217195
853.0 1768 0.4824660633484163
892.0 1768 0.504524886877828


In [60]:
np.savez('NEO_train', inputs=train_inputs, targets=train_targets)
np.savez('NEO_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('NEO_test', inputs=test_inputs, targets=test_targets)